머신러닝 > 데이터 패턴 인식

공부시간   2,    3,    7,  10
시험성적 70,  72,  80,  95

관계식을 생성, 입력 대비 출력에 대한 함수 --> 직선 방정식

회귀: 값의 흐름(값을 예측) 
이진분류: 확률(입력값의 결과가 1이 될 확률, sigmoid로 확률이 0.5보다 낮으면 0 아니면 1로 변환)
다항분류: 확률 (입력값의 결과가 해당 분류값이 될 확률, 각 종류별 확률이 반환되고 그 중 가장 높은 값으로 예측 - 모든 확률의 합이 1)
				a: 0.2, b: 0.5, c: 0.3
sklearn.linaer.LinearRegression
sklearn.neighbors.KneighborsClassifier
sklearn.neighbors.KneighborsRegressor

작업순서
1. 데이터 로딩
import pandas as pd
ti = pd.read_csv("./titanic.csv")

2. 타입확인
ti.info()

3. 독립변수, 종속변수 구분
y = ti["survived"]
X = ti[   [" .... "]   ] 

4. 독립변수에서 결측치 제거, 학습할 때 오류발생
ti.isna().sum()

ti["age"] = ti["age"].fillna( ti["age"].mean() )

In [1]:
# 1. 데이터 로딩
import pandas as pd
# ti = pd.read_csv("./titanic.csv")
import seaborn as sns
ti = sns.load_dataset("titanic")

# 2. 타입확인
ti.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [3]:
# 3. 독립변수, 종속변수 구분
y = ti["survived"]
X = ti[   ["pclass","age","sibsp","parch","fare"]   ] 

# 4. 독립변수에서 결측치 제거, 학습할 때 오류발생
ti.isna().sum()



survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# age의 결측치는 age의 평균으로 채움
X["age"] = ti["age"].fillna( ti["age"].mean() )
# 결측치 검사
X.isna().sum()

/tmp/ipykernel_4994/74648944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["age"] = ti["age"].fillna( ti["age"].mean() )


pclass    0
age       0
sibsp     0
parch     0
fare      0
dtype: int64

In [6]:
# X를 표준화 시킴, 각 독립변수들은 독립적인 정보를 가짐. 
# 모든 독립변수들이 같은 정보를 표현할 때엔 정규화 (MinMaxScaler)
from sklearn.preprocessing import StandardScaler
X_scale = StandardScaler().fit_transform(X)

In [7]:
# 훈련, 테스트데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_scale, y, random_state=0, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((668, 5), (668,), (223, 5), (223,))

In [9]:
# 머신러닝 (이항분류)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter = 1000)
model.fit(X_train,y_train)
model.score(X_train,y_train), model.score(X_test,y_test)

(0.7125748502994012, 0.6860986547085202)

In [12]:
# 딥러닝 (이항분류)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()  # 이 방법과 Sequential을 만들 때 Dense를 list형태로 넘겨주는 방법이 있다.(교재 p.18 참고)
model.add(
    Dense(10, activation = 'ReLU', # 칼럼이 5개이므로 대략 10개의 unit 생성
    input_shape = (X_train.shape[1],))
) 
model.add(
    Dense(1, activation = 'sigmoid')
)
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                60        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 71
Trainable params: 71
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 1개 정답과 예측값의 차이: 오차
# 모든 입력값의 오차의 합: 비용(cost, loss)
# 정답을 찾는 방법 정의
model.compile(
    loss='binary_crossentropy',  # 회귀: mse, 다항: (sparse_)categorical_crossentropy
    optimizer='Adam', # 오차에 대한 미분과 미분된 값을 W에 업데이트, 즉 back propagation을 자동화시켜주는 기능 (기울기 미분, 가중치 업데이트 자동화 도구)
    metrics=['Accuracy'] # 옵션 사항으로, 지정한 loss외의 값으로 성능 측정을 하고 싶을 때 명시. 리스트형식이기 때문에 여러개 명시 가능
)

In [19]:
# 조기종료 콜백 
from tensorflow.keras import callbacks
cb = callbacks.EarlyStopping(monitor='loss', patience=3)
# 모델 훈련
model.fit(X_train, y_train, epochs=100, verbose=1, callbacks=[cb]) # 콜백 역시 리스트로 여러 콜백 옵션을 줄 수 있다.
# 훈련이 덜 된 것 같을 땐 fit만 따로 빼서 돌림으로 정확도를 올릴 수 있다.
# 위 셀의 코드와 같이 돌리면 매번 새로운 모델로 초기화 된다.

Epoch 1/100
21/21 [==============================] - 0s 654us/step - loss: 0.5412 - Accuracy: 0.7395
Epoch 2/100
21/21 [==============================] - 0s 704us/step - loss: 0.5413 - Accuracy: 0.7395
Epoch 3/100
21/21 [==============================] - 0s 741us/step - loss: 0.5411 - Accuracy: 0.7380
Epoch 4/100
21/21 [==============================] - 0s 753us/step - loss: 0.5408 - Accuracy: 0.7380
Epoch 5/100
21/21 [==============================] - 0s 739us/step - loss: 0.5408 - Accuracy: 0.7395
Epoch 6/100
21/21 [==============================] - 0s 769us/step - loss: 0.5407 - Accuracy: 0.7395
Epoch 7/100
21/21 [==============================] - 0s 767us/step - loss: 0.5405 - Accuracy: 0.7395
Epoch 8/100
21/21 [==============================] - 0s 803us/step - loss: 0.5404 - Accuracy: 0.7380
Epoch 9/100
21/21 [==============================] - 0s 763us/step - loss: 0.5404 - Accuracy: 0.7380
Epoch 10/100
21/21 [==============================] - 0s 760us/step - loss: 0.5403 - Accura

21/21 [==============================] - 0s 752us/step - loss: 0.5356 - Accuracy: 0.7455


In [20]:
# 모델 성능 측정
model.evaluate(X_train, y_train), model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 778us/step - loss: 0.5735 - Accuracy: 0.6906


([0.5350239276885986, 0.7470059990882874],
 [0.5735021233558655, 0.6905829310417175])